# 📊 Data Analysis Notebook

This notebook performs **Exploratory Data Analysis (EDA)** on the cleaned car sales data.

**Datasets:**
- `Cars_cleaned.csv` - Car inventory data
- `Customers_cleaned.csv` - Customer information
- `Sales_cleaned.csv` - Sales transaction records

## 1. Imports and Setup

In [ ]:
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import sys
import os

# Add project root to path to allow importing src
from pathlib import Path
project_root = Path(os.getcwd())
while not (project_root / '.git').exists() and project_root != project_root.parent:
    project_root = project_root.parent
sys.path.append(str(project_root))

from src.utils.data_manager import load_from, save_to, save_result, ChartConfig

# Set styles
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

# Premium color palette
COLORS = ['#4c72b0', '#55a868', '#c44e52', '#8172b2', '#ccb974', '#64b5cd']
px.defaults.color_discrete_sequence = COLORS

print('✅ Libraries and Helper Functions loaded successfully!')

## 2. Load Data

In [ ]:
# Load datasets using helper functions
df_cars = pl.read_csv(load_from('cleaned', 'Cars_cleaned.csv'))
df_customers = pl.read_csv(load_from('cleaned', 'Customers_cleaned.csv'))
df_sales = pl.read_csv(load_from('cleaned', 'Sales_cleaned.csv'))

print(f'🚗 Cars: {df_cars.shape[0]:,} rows, {df_cars.shape[1]} columns')
print(f'👤 Customers: {df_customers.shape[0]:,} rows, {df_customers.shape[1]} columns')
print(f'💰 Sales: {df_sales.shape[0]:,} rows, {df_sales.shape[1]} columns')

In [ ]:
# Preview Cars data
print("=== Cars Dataset ===")
print(df_cars.schema)
df_cars.head()

In [ ]:
# Preview Customers data
print("=== Customers Dataset ===")
print(df_customers.schema)
df_customers.head()

In [ ]:
# Preview Sales data
print("=== Sales Dataset ===")
print(df_sales.schema)
df_sales.head()

## 3. Data Merging

In [ ]:
# Join Sales with Cars on Car_ID
df_master = df_sales.join(df_cars, on="Car_ID", how="left")

# Join with Customers on Customer_ID
df_master = df_master.join(df_customers, on="Customer_ID", how="left")

print(f"📊 Master Dataset: {df_master.shape[0]:,} rows, {df_master.shape[1]} columns")
print(f"\nColumns: {df_master.columns}")

In [ ]:
# Preview master dataset
df_master.head()

---
## 4. Exploratory Data Analysis (EDA)

### 4.1 Univariate Analysis

#### 4.1.1 Numerical Distributions

In [ ]:
# Price Distribution (Cars)
fig = px.histogram(
    df_cars.to_pandas(), 
    x="Price", 
    nbins=50,
    title="🚗 Car Price Distribution",
    labels={"Price": "Price ($)", "count": "Frequency"},
    color_discrete_sequence=[COLORS[0]]
)
fig.update_layout(bargap=0.1)
fig.show()

In [ ]:
# Age Distribution (Customers)
fig = px.histogram(
    df_customers.to_pandas(),
    x="Age",
    nbins=30,
    title="👤 Customer Age Distribution",
    labels={"Age": "Age (years)", "count": "Frequency"},
    color_discrete_sequence=[COLORS[2]]
)
fig.show()

#### 4.1.2 Categorical Frequencies

In [ ]:
# Top 10 Brands
brand_counts = df_cars.group_by("Brand").agg(pl.count().alias("Count")).sort("Count", descending=True).head(10)

fig = px.bar(
    brand_counts.to_pandas(),
    x="Brand",
    y="Count",
    title="🏷️ Top Car Brands (by Inventory Count)",
    color="Brand",
    color_discrete_sequence=COLORS
)
fig.update_layout(showlegend=False)
fig.show()

In [ ]:
# Top 10 Models
model_counts = df_cars.group_by("Model").agg(pl.count().alias("Count")).sort("Count", descending=True).head(10)

fig = px.bar(
    model_counts.to_pandas(),
    x="Model",
    y="Count",
    title="🚘 Top 10 Car Models (by Inventory Count)",
    color="Model",
    color_discrete_sequence=COLORS
)
fig.update_layout(showlegend=False)
fig.show()

In [ ]:
# Engine Type Distribution
engine_counts = df_cars.group_by("Engine_Type").agg(pl.count().alias("Count")).sort("Count", descending=True)

fig = px.pie(
    engine_counts.to_pandas(),
    values="Count",
    names="Engine_Type",
    title="⚙️ Engine Type Distribution",
    color_discrete_sequence=COLORS,
    hole=0.4
)
fig.show()

In [ ]:
# Transmission Distribution
trans_counts = df_cars.group_by("Transmission").agg(pl.count().alias("Count")).sort("Count", descending=True)

fig = px.pie(
    trans_counts.to_pandas(),
    values="Count",
    names="Transmission",
    title="🔧 Transmission Type Distribution",
    color_discrete_sequence=COLORS,
    hole=0.4
)
fig.show()

In [ ]:
# Region Distribution (Customers)
region_counts = df_customers.group_by("Region").agg(pl.count().alias("Count")).sort("Count", descending=True)

fig = px.bar(
    region_counts.to_pandas(),
    x="Region",
    y="Count",
    title="🗺️ Customer Distribution by Region",
    color="Region",
    color_discrete_sequence=COLORS
)
fig.update_layout(showlegend=False)
fig.show()

In [ ]:
# Payment Method Distribution (Sales)
payment_counts = df_sales.group_by("Payment_Method").agg(pl.count().alias("Count")).sort("Count", descending=True)

fig = px.pie(
    payment_counts.to_pandas(),
    values="Count",
    names="Payment_Method",
    title="💳 Payment Method Distribution",
    color_discrete_sequence=COLORS,
    hole=0.4
)
fig.show()

#### 4.1.3 Outlier Detection

In [ ]:
# Box plots for Price and Quantity
fig = make_subplots(rows=1, cols=2, subplot_titles=("Car Price", "Sale Quantity"))

fig.add_trace(
    go.Box(y=df_cars["Price"].to_list(), name="Price", marker_color=COLORS[0]),
    row=1, col=1
)

fig.add_trace(
    go.Box(y=df_sales["Quantity"].to_list(), name="Quantity", marker_color=COLORS[1]),
    row=1, col=2
)

fig.update_layout(title_text="📦 Outlier Detection - Box Plots", showlegend=False)
fig.show()

---
### 4.2 Bivariate & Multivariate Analysis

#### 4.2.1 Correlation Analysis

In [ ]:
# Select numerical columns for correlation
numerical_cols = ["Price", "Year", "Quantity_In_Stock", "Quantity", "Sale_Price", "Age"]

# Filter columns that exist in master dataset
available_cols = [col for col in numerical_cols if col in df_master.columns]

# Convert to pandas for correlation matrix
corr_df = df_master.select(available_cols).to_pandas()
corr_matrix = corr_df.corr()

# Heatmap
fig = px.imshow(
    corr_matrix,
    text_auto=".2f",
    title="🔥 Correlation Heatmap",
    color_continuous_scale="RdBu_r",
    zmin=-1, zmax=1
)
fig.show()

#### 4.2.2 Sales Performance by Category

In [ ]:
# Average Price per Brand
avg_price_brand = df_master.group_by("Brand").agg(
    pl.mean("Price").alias("Avg_Price")
).sort("Avg_Price", descending=True)

fig = px.bar(
    avg_price_brand.to_pandas(),
    x="Brand",
    y="Avg_Price",
    title="💰 Average Car Price by Brand",
    color="Brand",
    color_discrete_sequence=COLORS
)
fig.update_layout(showlegend=False, yaxis_title="Average Price ($)")
fig.show()

In [ ]:
# Average Price per Engine Type
avg_price_engine = df_master.group_by("Engine_Type").agg(
    pl.mean("Sale_Price").alias("Avg_Sale_Price")
).sort("Avg_Sale_Price", descending=True)

fig = px.bar(
    avg_price_engine.to_pandas(),
    x="Engine_Type",
    y="Avg_Sale_Price",
    title="⚙️ Average Sale Price by Engine Type",
    color="Engine_Type",
    color_discrete_sequence=COLORS
)
fig.update_layout(showlegend=False, yaxis_title="Average Sale Price ($)")
fig.show()

In [ ]:
# --- 1. Data Aggregation ---
# Group by car model to get the required metrics
model_sales_summary = df_master.group_by("Model").agg(
    pl.sum("Quantity").alias("Total_Quantity_Sold"),
    pl.mean("Sale_Price").alias("Avg_Sale_Price"),
    pl.count().alias("Transaction_Count")
).sort("Model")

# Convert to pandas for easier plotting with Plotly GO
model_sales_summary_pd = model_sales_summary.to_pandas()


# --- 2. Create the 3-Chart Layout ---
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=(
        "<b>Top 15 Models by Quantity Sold</b>",
        "<b>Correlation: Quantity Sold vs. Average Price</b>",
        "<b>Top 15 Models by Average Price</b>"
    ),
    column_widths=[0.28, 0.44, 0.28] # Give center plot more space
)


# --- 3. Left Chart: Top Models by Quantity Sold ---
top_quantity = model_sales_summary_pd.sort_values("Total_Quantity_Sold", ascending=False).head(15)
fig.add_trace(go.Bar(
    x=top_quantity['Model'],
    y=top_quantity['Total_Quantity_Sold'],
    marker_color=COLORS[1],
    name="Quantity Sold"
), row=1, col=1)


# --- 4. Middle Chart: Correlation Scatter Plot ---
fig.add_trace(go.Scatter(
    x=model_sales_summary_pd['Total_Quantity_Sold'],
    y=model_sales_summary_pd['Avg_Sale_Price'],
    mode='markers',
    text=model_sales_summary_pd['Model'], # Text for hover
    hovertemplate=(
        "<b>%{text}</b><br><br>" +
        "Total Units Sold: %{x:,}<br>" +
        "Average Sale Price: $%{y:,.2f}" +
        "<extra></extra>" # Hides the trace name on hover
    ),
    marker=dict(color=COLORS[0], size=8, opacity=0.7)
), row=1, col=2)


# --- 5. Right Chart: Top Models by Average Price ---
top_price = model_sales_summary_pd.sort_values("Avg_Sale_Price", ascending=False).head(15)
fig.add_trace(go.Bar(
    x=top_price['Model'],
    y=top_price['Avg_Sale_Price'],
    marker_color=COLORS[2],
    name="Average Price"
), row=1, col=3)


# --- 6. Final Layout Updates ---
fig.update_layout(
    title_text="<b>Analysis of Sales Volume vs. Average Price by Car Model</b>",
    height=600,
    showlegend=False,
    template="plotly_white",
    bargap=0.2
)

# Update axis titles for clarity
fig.update_yaxes(title_text="Total Units Sold", row=1, col=1)
fig.update_xaxes(title_text="Car Model", tickangle=-45, row=1, col=1)

fig.update_xaxes(title_text="Total Units Sold", row=1, col=2)
fig.update_yaxes(title_text="Average Sale Price ($)", row=1, col=2)

fig.update_yaxes(title_text="Average Sale Price ($)", row=1, col=3)
fig.update_xaxes(title_text="Car Model", tickangle=-45, row=1, col=3)

fig.show()

In [ ]:
# Sales Volume by Region
sales_by_region = df_master.group_by("Region").agg(
    pl.sum("Quantity").alias("Total_Quantity"),
    pl.sum("Sale_Price").alias("Total_Revenue")
).sort("Total_Revenue", descending=True)

fig = px.bar(
    sales_by_region.to_pandas(),
    x="Region",
    y="Total_Revenue",
    title="🗺️ Total Revenue by Region",
    color="Region",
    color_discrete_sequence=COLORS
)
fig.update_layout(showlegend=False, yaxis_title="Total Revenue ($)")
fig.show()

In [ ]:
# Sales by Payment Method
sales_by_payment = df_master.group_by("Payment_Method").agg(
    pl.sum("Sale_Price").alias("Total_Revenue"),
    pl.count().alias("Transaction_Count")
).sort("Total_Revenue", descending=True)

fig = px.bar(
    sales_by_payment.to_pandas(),
    x="Payment_Method",
    y="Total_Revenue",
    title="💳 Total Revenue by Payment Method",
    color="Payment_Method",
    color_discrete_sequence=COLORS
)
fig.update_layout(showlegend=False, yaxis_title="Total Revenue ($)")
fig.show()

In [ ]:
# Top 10 Salespersons by Revenue
top_salespersons = df_master.group_by("Salesperson").agg(
    pl.sum("Sale_Price").alias("Total_Revenue"),
    pl.count().alias("Deals_Closed")
).sort("Total_Revenue", descending=True).head(10)

fig = px.bar(
    top_salespersons.to_pandas(),
    x="Salesperson",
    y="Total_Revenue",
    title="🏆 Top 10 Salespersons by Revenue",
    color="Salesperson",
    color_discrete_sequence=COLORS
)
fig.update_layout(showlegend=False, yaxis_title="Total Revenue ($)", xaxis_tickangle=-45)
fig.show()

#### 4.2.3 Scatter Plot Matrix

In [ ]:
# Scatter matrix for key numerical features
scatter_cols = ["Price", "Year", "Sale_Price", "Quantity", "Age"]
scatter_cols = [col for col in scatter_cols if col in df_master.columns]

fig = px.scatter_matrix(
    df_master.select(scatter_cols).sample(n=min(1000, df_master.shape[0])).to_pandas(),
    dimensions=scatter_cols,
    title="🔍 Scatter Plot Matrix",
    color_discrete_sequence=[COLORS[0]]
)
fig.update_traces(diagonal_visible=False)
fig.update_layout(height=800)
fig.show()

---
### 4.3 Temporal Analysis (Time-Series)

In [ ]:
# Parse Sale_Date to date type
df_master = df_master.with_columns(
    pl.col("Sale_Date").str.to_date().alias("Sale_Date_Parsed")
)

# Extract Month and Day of Week
df_master = df_master.with_columns([
    pl.col("Sale_Date_Parsed").dt.month().alias("Month"),
    pl.col("Sale_Date_Parsed").dt.weekday().alias("DayOfWeek"),
    pl.col("Sale_Date_Parsed").dt.year().alias("Sale_Year")
])

print("✅ Date features extracted!")

In [ ]:
# Monthly Sales Trend
monthly_sales = df_master.group_by(["Sale_Year", "Month"]).agg(
    pl.sum("Sale_Price").alias("Total_Revenue"),
    pl.count().alias("Transaction_Count")
).sort(["Sale_Year", "Month"])

# Create a combined date column for plotting
monthly_sales = monthly_sales.with_columns(
    (pl.col("Sale_Year").cast(str) + "-" + pl.col("Month").cast(str).str.zfill(2)).alias("Year_Month")
)

fig = px.line(
    monthly_sales.to_pandas(),
    x="Year_Month",
    y="Total_Revenue",
    title="📈 Monthly Revenue Trend",
    markers=True,
    color_discrete_sequence=[COLORS[0]]
)
fig.update_layout(xaxis_title="Month", yaxis_title="Total Revenue ($)", xaxis_tickangle=-45)
fig.show()

In [ ]:
# Sales by Month (Seasonality)
month_names = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

seasonality = df_master.group_by("Month").agg(
    pl.sum("Sale_Price").alias("Total_Revenue"),
    pl.count().alias("Transaction_Count")
).sort("Month")

seasonality_pd = seasonality.to_pandas()
seasonality_pd["Month_Name"] = seasonality_pd["Month"].apply(lambda x: month_names[x-1] if 1 <= x <= 12 else "Unknown")

fig = px.bar(
    seasonality_pd,
    x="Month_Name",
    y="Total_Revenue",
    title="📆 Seasonality: Revenue by Month",
    color="Total_Revenue",
    color_continuous_scale="Blues"
)
fig.update_layout(xaxis_title="Month", yaxis_title="Total Revenue ($)")
fig.show()

In [ ]:
# Sales by Day of Week
day_names = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]

dow_sales = df_master.group_by("DayOfWeek").agg(
    pl.sum("Sale_Price").alias("Total_Revenue"),
    pl.count().alias("Transaction_Count")
).sort("DayOfWeek")

dow_pd = dow_sales.to_pandas()
dow_pd["Day_Name"] = dow_pd["DayOfWeek"].apply(lambda x: day_names[x] if 0 <= x <= 6 else "Unknown")

fig = px.bar(
    dow_pd,
    x="Day_Name",
    y="Total_Revenue",
    title="📅 Sales by Day of Week",
    color="Total_Revenue",
    color_continuous_scale="Greens"
)
fig.update_layout(xaxis_title="Day of Week", yaxis_title="Total Revenue ($)")
fig.show()

---
### 4.4 Data Integrity Check

In [ ]:
# Data Shape Verification
print("=== Data Shape Verification ===")
print(f"Original Sales rows: {df_sales.shape[0]:,}")
print(f"Master Dataset rows: {df_master.shape[0]:,}")
print(f"Difference: {df_sales.shape[0] - df_master.shape[0]:,} rows")

if df_sales.shape[0] == df_master.shape[0]:
    print("\n✅ No data loss during merge!")
else:
    print("\n⚠️ Some rows may have been lost during merge. Check for missing IDs.")

In [ ]:
# Cross-Table Consistency: Check for Car_IDs in Sales not in Cars
sales_car_ids = set(df_sales["Car_ID"].to_list())
cars_car_ids = set(df_cars["Car_ID"].to_list())

missing_cars = sales_car_ids - cars_car_ids

print("=== Car_ID Integrity Check ===")
print(f"Unique Car_IDs in Sales: {len(sales_car_ids):,}")
print(f"Unique Car_IDs in Cars: {len(cars_car_ids):,}")
print(f"Car_IDs in Sales but not in Cars: {len(missing_cars):,}")

if len(missing_cars) == 0:
    print("\n✅ All Car_IDs in Sales exist in Cars table!")
else:
    print(f"\n⚠️ {len(missing_cars)} Car_IDs in Sales are not in Cars table:")
    print(list(missing_cars)[:10])  # Show first 10

In [ ]:
# Cross-Table Consistency: Check for Customer_IDs in Sales not in Customers
sales_customer_ids = set(df_sales["Customer_ID"].to_list())
customers_customer_ids = set(df_customers["Customer_ID"].to_list())

missing_customers = sales_customer_ids - customers_customer_ids

print("=== Customer_ID Integrity Check ===")
print(f"Unique Customer_IDs in Sales: {len(sales_customer_ids):,}")
print(f"Unique Customer_IDs in Customers: {len(customers_customer_ids):,}")
print(f"Customer_IDs in Sales but not in Customers: {len(missing_customers):,}")

if len(missing_customers) == 0:
    print("\n✅ All Customer_IDs in Sales exist in Customers table!")
else:
    print(f"\n⚠️ {len(missing_customers)} Customer_IDs in Sales are not in Customers table:")
    print(list(missing_customers)[:10])  # Show first 10

In [ ]:
# Save merged dataset to cleaned folder using helper function
merged_path = save_to('cleaned', 'Sales_merged.csv')
df_master.write_csv(merged_path)
print(f'✅ Merged dataset saved to: {merged_path}')
print(f'   Shape: {df_master.shape[0]:,} rows × {df_master.shape[1]} columns')

In [ ]:
# --- Publish Highlights to Dashboard ---
total_revenue = df_master['Sale_Price'].sum()
total_units = df_master.shape[0]
avg_price = df_master['Sale_Price'].mean()
top_region = df_master.group_by('Region').agg(pl.sum('Sale_Price')).sort('Sale_Price', descending=True)['Region'][0]

summary_data = {
    'metrics': {
        'total_revenue': round(total_revenue, 2),
        'total_units': total_units,
        'average_price': round(avg_price, 2),
        'top_performing_region': top_region
    },
    'last_updated': '2026-01-21'
}

save_result(summary_data, 'analysis_summary.toml', topic='analysis')

In [ ]:
# Create and publish regional performance chart config
regional_data = df_master.group_by('Region').agg(
    pl.sum('Sale_Price').alias('Revenue'),
    pl.sum('Quantity').alias('Units')
).sort('Revenue', descending=True).to_dicts()

chart = ChartConfig(
    title='Regional Sales Performance',
    chart_type='bar',
    description='Revenue and Units sold by region',
    x_axis_key='Region'
)
chart.add_series('Revenue', 'Revenue ($)', color=COLORS[0])
chart.add_series('Units', 'Units Sold', color=COLORS[1])
chart.set_data(regional_data)

save_result(chart, 'regional_performance', topic='analysis', file_format='toml')

In [ ]:
# Create and publish monthly trend chart config
trend_data = monthly_sales.select(['Year_Month', 'Total_Revenue']).to_dicts()

chart = ChartConfig(
    title='Monthly Revenue Trend',
    chart_type='area',
    description='Monthly revenue growth over time',
    x_axis_key='Year_Month'
)
chart.add_series('Total_Revenue', 'Monthly Revenue ($)', color=COLORS[3])
chart.set_data(trend_data)

save_result(chart, 'monthly_trend', topic='analysis', file_format='toml')

---
## ✅ Analysis Complete

This notebook has performed:
1. **Univariate Analysis** - Distributions and frequencies
2. **Bivariate & Multivariate Analysis** - Correlations and performance metrics
3. **Temporal Analysis** - Trends and seasonality
4. **Data Integrity Checks** - Cross-table consistency verification